In [1]:
import sys
!{sys.executable} -m pip install neo4j==5.0.1

In [23]:
# Import the neo4j dependency
from neo4j import GraphDatabase

uri = "neo4j://corso-kg-neo4j:7687"

username = 'neo4j'
password = 'kgCefriel'
auth = (username, password)

# Create a new Driver instance
driver = GraphDatabase.driver(uri,auth=auth)
driver.verify_connectivity()

In [25]:
from os import listdir, getcwd
from os.path import abspath, basename, isfile, join, dirname

folder = dirname(getcwd())
import_path = join(folder, "data")

with driver.session() as session:
    def create_constraint_competences(tx):
        return tx.run("""
            CREATE CONSTRAINT CompetenceId IF NOT EXISTS ON (c:Competence) ASSERT c.competenceId IS UNIQUE
            """)
    def create_constraint_users(tx):
        return tx.run("""
            CREATE CONSTRAINT EmployeeId IF NOT EXISTS ON (e:Employee) ASSERT e.employeeId IS UNIQUE
            """)
    print("Create constraints")
    session.execute_write(create_constraint_competences)
    session.execute_write(create_constraint_users)

    def create_competences(tx):
        return tx.run("""
            LOAD CSV WITH HEADERS
            FROM "file:///competences.csv"
            AS row
            MERGE (c:Competence {competenceId: row.id})
            ON CREATE SET
            c.dimension = row.dimension,
            c.source = row.source,
            c.competence = row.competence,
            c.description = row.description
            """)
    print("Create competences")
    session.execute_write(create_competences)

    def create_units(tx):
        return tx.run("""
            LOAD CSV WITH HEADERS
            FROM "file:///employees.csv"
            AS row
            MERGE (u:Unit {unitName: row.unit})
            """)
    print("Create units")
    session.execute_write(create_units)

    def create_employees(tx):
        return tx.run("""
            LOAD CSV WITH HEADERS
            FROM "file:///employees.csv"
            AS row
            MERGE (e:Employee {employeeId: row.id})
            ON CREATE SET
            e.name = row.name,
            e.surname = row.surname
            WITH e, row
            MATCH (u:Unit {unitName: row.unit})
            MERGE (e)-[r:IN_UNIT]->(u)
            """)
    print("Create employees")
    session.execute_write(create_employees)

    def load_assignments(tx):
        cypher = """
            LOAD CSV WITH HEADERS
            FROM "file:///assignment.csv"
            AS row
            MATCH (e:Employee {employeeId: row.employee_id})
            MATCH (c:Competence {competenceId: row.competence_id})
            MERGE (e)-[r:HAS_COMPETENCE]->(c)
            ON CREATE SET r.level = row.level
        """
        return tx.run(cypher)
    print("Load assignments")
    session.execute_write(load_assignments)
    
    def load_interest(tx):
        cypher = """
            LOAD CSV WITH HEADERS
            FROM "file:///assignment.csv"
            AS row
            WITH row WHERE row.interest = "1"
            MATCH (e:Employee {employeeId: row.employee_id})
            MATCH (c:Competence {competenceId: row.competence_id})
            MERGE (e)-[r:INTERESTED_IN]->(c)
        """
        return tx.run(cypher)
    print("Load interest")
    session.execute_write(load_interest)

Create constraints
Create competences
Create units
Create employees
Load assignments
Load interest


In [12]:
def execute_query(custom_query):
    # Execute the custom query
    with driver.session() as session:
        result = session.run(custom_query)
    
        # Extract the column names and rows from the query result
        columns = result.keys()
        rows = result.data()
    
        # Print the custom query results as a table
        for col in columns:
            print(col, end="\t")
        print()
        for row in rows:
            for col in columns:
                print(row[col], end="\t")
            print()

In [22]:
# Other employees having same competences
custom_query = """
    MATCH (e:Employee {name:"Jane"})
    RETURN e.name AS name, e.surname AS surname
"""
execute_query(custom_query)

name	surname	
Jane	Wilson	
Jane	Smith	
Jane	Brown	


In [16]:
# Other employees having the same competences
custom_query = """
    MATCH (e:Employee {name:"Jane", surname:"Wilson"})-[:HAS_COMPETENCE]->(c:Competence)<-[:HAS_COMPETENCE]-(oe:Employee) 
    RETURN c.competence AS competence, collect(oe.employeeId) AS employees
"""
execute_query(custom_query)

competence	employees	
Netiquette	['EMPLOYEE_20', 'EMPLOYEE_12', 'EMPLOYEE_21', 'EMPLOYEE_8', 'EMPLOYEE_17', 'EMPLOYEE_19', 'EMPLOYEE_2', 'EMPLOYEE_13']	
Developing digital content	['EMPLOYEE_28', 'EMPLOYEE_12', 'EMPLOYEE_18', 'EMPLOYEE_25', 'EMPLOYEE_2', 'EMPLOYEE_29', 'EMPLOYEE_6', 'EMPLOYEE_15', 'EMPLOYEE_9', 'EMPLOYEE_23']	
Protecting personal data and privacy	['EMPLOYEE_19', 'EMPLOYEE_14', 'EMPLOYEE_22', 'EMPLOYEE_17', 'EMPLOYEE_4', 'EMPLOYEE_18', 'EMPLOYEE_21', 'EMPLOYEE_25', 'EMPLOYEE_12', 'EMPLOYEE_26', 'EMPLOYEE_11', 'EMPLOYEE_9', 'EMPLOYEE_24', 'EMPLOYEE_29', 'EMPLOYEE_7', 'EMPLOYEE_20', 'EMPLOYEE_6', 'EMPLOYEE_2']	
Interacting through digital technologies	['EMPLOYEE_16', 'EMPLOYEE_13', 'EMPLOYEE_20', 'EMPLOYEE_3', 'EMPLOYEE_27', 'EMPLOYEE_21', 'EMPLOYEE_8', 'EMPLOYEE_11', 'EMPLOYEE_30', 'EMPLOYEE_1', 'EMPLOYEE_9']	


In [19]:
# More "similar" employees
custom_query = """
    MATCH (e:Employee {name:"Jane", surname:"Wilson"})-[:HAS_COMPETENCE]->(c:Competence)<-[:HAS_COMPETENCE]-(oe:Employee) 
    RETURN oe.employeeId AS employee, count(c) as num ORDER BY num DESC
"""
execute_query(custom_query)

employee	num	
EMPLOYEE_20	3	
EMPLOYEE_12	3	
EMPLOYEE_21	3	
EMPLOYEE_2	3	
EMPLOYEE_9	3	
EMPLOYEE_8	2	
EMPLOYEE_17	2	
EMPLOYEE_19	2	
EMPLOYEE_13	2	
EMPLOYEE_18	2	
EMPLOYEE_25	2	
EMPLOYEE_29	2	
EMPLOYEE_6	2	
EMPLOYEE_11	2	
EMPLOYEE_28	1	
EMPLOYEE_15	1	
EMPLOYEE_23	1	
EMPLOYEE_14	1	
EMPLOYEE_22	1	
EMPLOYEE_4	1	
EMPLOYEE_26	1	
EMPLOYEE_24	1	
EMPLOYEE_7	1	
EMPLOYEE_16	1	
EMPLOYEE_3	1	
EMPLOYEE_27	1	
EMPLOYEE_30	1	
EMPLOYEE_1	1	


In [ ]:
# Close the Neo4j driver
driver.close()